## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
0,Lebu,-37.6167,-73.6500,62.33,67,28,8.50,CL,scattered clouds,2022-01-22 18:42:03
1,Caravelas,-17.7125,-39.2481,82.76,65,14,9.33,BR,few clouds,2022-01-22 18:42:03
2,Ancud,-41.8697,-73.8203,62.13,70,67,16.91,CL,light rain,2022-01-22 18:42:03
3,Jamestown,42.0970,-79.2353,23.09,34,0,7.00,US,clear sky,2022-01-22 18:39:47
4,Puerto Ayora,-0.7393,-90.3518,80.56,75,85,7.90,EC,overcast clouds,2022-01-22 18:42:04


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
0,Lebu,-37.6167,-73.6500,62.33,67,28,8.50,CL,scattered clouds,2022-01-22 18:42:03
1,Caravelas,-17.7125,-39.2481,82.76,65,14,9.33,BR,few clouds,2022-01-22 18:42:03
2,Ancud,-41.8697,-73.8203,62.13,70,67,16.91,CL,light rain,2022-01-22 18:42:03
4,Puerto Ayora,-0.7393,-90.3518,80.56,75,85,7.90,EC,overcast clouds,2022-01-22 18:42:04
8,Hilo,19.7297,-155.0900,74.62,75,0,1.99,US,clear sky,2022-01-22 18:39:10
9,Biak,-0.9131,122.8766,75.27,84,61,3.65,ID,broken clouds,2022-01-22 18:42:05
11,Dicabisagan,17.0818,122.4157,74.93,85,90,5.53,PH,overcast clouds,2022-01-22 18:42:06
13,Pisco,-13.7000,-76.2167,77.05,65,0,9.22,PE,clear sky,2022-01-22 18:42:06
14,Rikitea,-23.1203,-134.9692,78.28,70,100,10.51,PF,overcast clouds,2022-01-22 18:42:07
15,Caconde,-21.5294,-46.6439,69.66,64,7,2.57,BR,clear sky,2022-01-22 18:42:07


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   325
Lat                    325
Lng                    325
Max Temp               325
Humidity               325
Cloudiness             325
Wind Speed             325
Country                323
Weather Description    325
Date                   325
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                2
Weather Description    0
Date                   0
dtype: int64

In [7]:
is_NaN = preferred_cities_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = preferred_cities_df[row_has_NaN]
rows_with_NaN
# Source: https://www.kite.com/python/answers/how-to-find-rows-with-nan-values-in-a-pandas-dataframe-in-python

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
139,Luderitz,-26.6481,15.1594,81.46,58,100,11.95,NaN,light rain,2022-01-22 18:45:43
321,Aranos,-24.1333,19.1167,83.07,25,100,13.15,NaN,overcast clouds,2022-01-22 18:50:37


In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')

In [9]:
clean_df.count()

City                   323
Lat                    323
Lng                    323
Max Temp               323
Humidity               323
Cloudiness             323
Wind Speed             323
Country                323
Weather Description    323
Date                   323
dtype: int64

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy() 

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Lebu,CL,62.33,scattered clouds,-37.6167,-73.6500,
1,Caravelas,BR,82.76,few clouds,-17.7125,-39.2481,
2,Ancud,CL,62.13,light rain,-41.8697,-73.8203,
4,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,
8,Hilo,US,74.62,clear sky,19.7297,-155.0900,
9,Biak,ID,75.27,broken clouds,-0.9131,122.8766,
11,Dicabisagan,PH,74.93,overcast clouds,17.0818,122.4157,
13,Pisco,PE,77.05,clear sky,-13.7000,-76.2167,
14,Rikitea,PF,78.28,overcast clouds,-23.1203,-134.9692,
15,Caconde,BR,69.66,clear sky,-21.5294,-46.6439,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
            print("Hotel not found... let's find another.")
        

Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's find another.
Hotel not found... let's 

In [24]:
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Lebu,CL,62.33,scattered clouds,-37.6167,-73.6500,Hostal Las Lilas
1,Caravelas,BR,82.76,few clouds,-17.7125,-39.2481,Pousada dos Navegantes
2,Ancud,CL,62.13,light rain,-41.8697,-73.8203,Hotel Arena Gruesa
4,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Hilo,US,74.62,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
9,Biak,ID,75.27,broken clouds,-0.9131,122.8766,Penginapan Kim
11,Dicabisagan,PH,74.93,overcast clouds,17.0818,122.4157,Nancy Montanes Bernardo
13,Pisco,PE,77.05,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
14,Rikitea,PF,78.28,overcast clouds,-23.1203,-134.9692,People ThankYou
15,Caconde,BR,69.66,clear sky,-21.5294,-46.6439,HOTEL - HOSPEDAGEM EM CACONDE


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
# No rows to drop

In [26]:
clean_hotel_df.count()

City                   323
Country                323
Max Temp               323
Weather Description    323
Lat                    323
Lng                    323
Hotel Name             323
dtype: int64

In [27]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
locations.head()

,Lat,Lng
0,-37.6167,-73.6500
1,-17.7125,-39.2481
2,-41.8697,-73.8203
4,-0.7393,-90.3518
8,19.7297,-155.0900


In [30]:
fig = gmaps.figure()

In [31]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))